In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd
import json

In [ ]:
from helper import check_5_vals, stricter_pm, get_uniques, Conn

In [ ]:
os.environ["PGDATABASE"] = "dil_login"

with open("../../database.env") as f:
    for line in f:
        key, value = line.strip().split('=', 1)
        os.environ[key] = value

In [ ]:
conn = Conn()

In [ ]:
tuples = conn.select("SELECT * FROM site")

sites = pd.DataFrame(tuples, columns=["id", "rank", "site", "urls", "crawl_urls", "timeout_crawl", "error", "error_py", "crawled_urls", "after_basic", "after_trees", "after_trees_limit", "actual_urls", "insertion_time", "confirmed_urls", "timeout_dyn", "finished", "login_urls"]).sort_values("rank")

display(sites.head())

tuples = conn.select("SELECT * FROM accept")
accept = pd.DataFrame(tuples, columns=["id", "site", "rank", "browser", "version", "clicked_count", "clicked", "locator_count", "unique_locators", "locators", "cookies_before", "cookies_after", "cookies_new", "cookies_removed", "cookies_changed", "error", "insertion_time"]).sort_values("rank")
display(accept.head())

more = False
if more:
    tuples = conn.select("SELECT * FROM dyn_conf")
    dyn_conf = pd.DataFrame(tuples, columns=["id", "browser", "version", "site", "opg_url", "url", "inc_method", "state", "run", "observation", "error", "notes", "response", "insertion_time"])
    display(dyn_conf.head())

    tuples = conn.select("SELECT * FROM responses")
    resp = pd.DataFrame(tuples, columns=["id", "site", "url", "state", "req_headers", "resp_code", "resp_headers",
                                           "resp_body_hash", "resp_body_info", "frames", "error_text",
                                           "insertion_time"])
    display(resp.head())

# Results

## 100 top websites with successful login

1. **URL Collection**:
    - Visit homepage (https://{site}/) wait until "load" (max: 30s) in Chromium
    - Extract all HTTP(S) links
    - Record all outgoing HTTP(S) requests


In [ ]:
# Some entries are duplicated (started several times, error in first data collection)
# Only keep the last entry
sites = sites.sort_values("id")
sites = sites.drop_duplicates(subset="rank", keep="last")

In [ ]:
sites["crawl_urls"] = sites["crawl_urls"].apply(lambda x: sorted(x))
sites["crawled_urls"] = sites["crawled_urls"].apply(lambda x: sorted(x))
sites["crawled_any"] = sites["crawled_urls"].str.len() != 0

In [ ]:
# Remove duplicates
print(sites["site"].nunique())
with pd.option_context("display.max_rows", 123):
    display(sites.sort_values(["site", "rank"])[["rank", "site"]])
    
# amazon.X; individual instances of amazon not exactly the same

# bbc.co.uk redirects to bbc.com, remove bbc.co.uk
sites = sites[sites["site"] != "bbc.co.uk"]

# bit.ly redirects to bitly.com, remove bit.ly
sites = sites[sites["site"] != "bit.ly"]

# google.com and google.com.hk; not exactly the same

# wikipedia.org and wikimedia.org; not exactly the same

# huffingtonpost.com redirects to huffpost.com; however lower rank; remove huffpost.com
sites = sites[sites["site"] != "huffpost.com"]

In [ ]:
# Overview of table structure
print("Overview:")
display(sites.head(2))

# Errors on the tested sites (URL + response collection)
print("Crawled sites:")
display(sites["crawled_any"].value_counts())
print("Errors on tested sites (crawled-any):")
display(sites[["crawled_any", "error"]].apply(lambda x: (x["crawled_any"], x["error"].split("\n")[0].split(" at ")[0]), axis=1).value_counts().to_frame())

display(sites["error_py"].value_counts().to_frame())

# One site not succesfully crawled:  nature.com, could not load landing page
display(sites.loc[~sites["crawled_any"]])

# One site crashed the infrastructure fandom.com, remove from analysis as data for fandom.com is lost
display(sites.loc[sites["error_py"].str.contains("ForkPool")])
sites = sites.loc[sites["site"] != "fandom.com"]


# URLs collected:
print(f"URLs collected on: {sites.loc[sites['urls'].str.len() != 0].shape[0]} sites")

print(f"URLs attempted to crawl on: {sites.loc[sites['crawl_urls'].str.len() != 0].shape[0]} sites")

# URLs crawled:
print(f"URLs crawled on: {sites.loc[sites['crawled_urls'].str.len() != 0].shape[0]} sites (the ones that are missing here crashed in collect_responses)")

# Same URLs crawled as tried:
print(f"All wanted URLs crawled on: {sites.loc[(sites['crawled_urls'] == sites['crawl_urls']) & (sites['crawled_urls'].str.len() != 0)].shape[0]} sites (either timeout or othe issue occured, e.g., crash in collect_responses)")



# Limit to crawled any sites
sites_crawled = sites.loc[sites["crawled_any"]]
print("Timeouts:")
display(sites_crawled[["timeout_crawl"]].value_counts().to_frame())
display(sites_crawled[["timeout_dyn"]].value_counts().to_frame())
display(sites_crawled[["timeout_crawl", "timeout_dyn"]].value_counts().to_frame())

2. **Response Collection**:
    - Open two chromium instances
    - visited homepage state **_visited_l** + login state **_login** (provided externally)
    - Visit URLs in all states (2)
        - on every URL wait until "load" (max: 30s) (top-level request)
        - max 1000 URLs (if more than 1000 exist, random selection of all recorded URLs)
        - max 1 hour
        - Record traffic/responses (with playwright; does not record everything for errors and similar; other option would be HAR or proxy?)

In [ ]:
def count(row):
    ll = row["urls"]
    links = []
    requests = []
    total = len(ll)
    for l in ll:
        if l["link"]:
            links.append(l)
        if l["request"]:
            requests.append(l)
    return {"Links": len(links), "Requests": len(requests), "Total": total}

print("URLs collected stats:")
display(sites_crawled.apply(count, result_type="expand", axis=1).describe())
display(sites_crawled.apply(count, result_type="expand", axis=1).sum())

3. **Pruning**:
    - Get all traffic data for all crawled URLs
    - Fit response data to trees:
        - Status-Code
        - smoothed (Security)-Headers: "content-type", "x-frame-options", "location", "content-disposition", "x-content-type-options", "cross-origin-opener-policy", "cross-origin-resource-policy", "content-security-policy"
        - body type: e.g., HTML, img, ... (inferred with `file` command)
    - Basic pruning: only keep URLs that have at least one attribute with more than one recorded value
    - Advanced pruning:
        - All Chromium and Firefox trees
        - Predict the outcome of every tree for every remaining URL-state pair
        - For every tree with at least two different predictions for a URL -> add URL-inclusion method to set of to confirm URLs
            - special cases for some trees (e.g., img-height):
                - even if all predictions are the same, they might be distinguishable (artifact of the smoothing)
                - if all predictions are positive (e.g, height=50), compare other property (e.g., bodyhash) and if that property differs -> add to set
            - example:     `urls = {"img": {"https://google.com/": "cfw"}, "https://google.com/search/": "c"}, "iframe": {"https://google.com/": "f"}}`; every inc-url pair is tested in both browsers regardless of prediction

In [ ]:
# sites crawled
sc = sites_crawled
def get_urls(dat):
    url_set = set()
    url_list = []
    for inc, entry in dat.items():
        for url, browser in entry.items():
            url_list.append(url)
            url_set.add(url)
    return url_list, url_set

def count_pruning(row):
    pc = row["crawl_urls"]
    c = row["crawled_urls"]
    ab = row["after_basic"]
    at = row["after_trees"]
    atl = row["after_trees_limit"]
    act = row["actual_urls"]
    ul_at, us_at = get_urls(at)
    ul_atl, us_atl = get_urls(atl)
    ul_act, us_act = get_urls(act)
                
    return {"crawl_urls": len(pc), "crawled_urls": len(c), "after basic": len(ab), "after trees (total inc-url pairs)": len(ul_at), "after trees (unique urls)": len(us_at), "after trees limit (total inc-url pairs)": len(ul_atl), "after trees limit (unique urls)": len(us_atl), "actual URLs": len(ul_act), "actual URLs (unique)": len(us_act)}
# The data describes it without browsers!
print("Pruning stats:")
display(sc[["crawl_urls", "crawled_urls", "after_basic", "after_trees", "after_trees_limit", "actual_urls"]].apply(count_pruning, axis=1, result_type="expand").describe())

In [ ]:
# Which inclusion methods are predicted?
# Sites/URLs
def collect_incs(row):
    row = row["after_trees"]
    res = {"any": {}}
    for inc in row.keys():
        for url, browser_str in row[inc].items():
            entry = res["any"].get(inc, 0)
            entry += 1
            res["any"][inc] = entry
    return res
met = sc[["after_trees"]].apply(collect_incs, axis=1, result_type="expand")
met_any = pd.json_normalize(met["any"]).agg(["count", "sum"]).T
met_any[["count", "sum"]]

4. **"Dynamic confirmation**:
    - Test all remaining inclusion_method-url-browser pairs
        - max 25 URLs for one inclusion method
        - max 3h
    - Test all possible states (regardless of whether the prediction was only for one state-pair)
    - Prepare states: 
        - Same as in **response collection**
        - Additionally for Firefox
    - For every inc method:
        - For every URL:
            - For every browser; If browser should be tested:
                - For every state:
                    - wait 1s
                    - visit `http://observer.org/opg/<inc>/?url=<url>`
                    - wait until "networkidle", max: 30s; for window.open wait for "networkidle" or "domcontentloaded" of the new window
                    - wait another 750ms (2000ms)
                    - extract observations
                    - (record responses)
                - If observations for every state are the same -> remove browser from to_test list
         - Repeat up to 5 times
     - Get confirmed distinguishable pairs:
         - 5 times different observations for one observation method -> confirmed browser-inc_method-url-state_a-state_b(-observation_method) pair
         - additional sanity checking: 
             - the same observation is not allowed to be present in both states (e.g., random frame counts: [(0, 1), (0, 1), (1, 0), (1, 0), (0, 1)] -> not a confirmed pair)
             - additional constraints for some methods: 
                 - e.g., custom code for postMessage, frame_count
                 - heuristic: at least one value should occur two times for the same state?

In [ ]:
# Timout of dynamic sites
sites_dyn = sc.loc[sc["actual_urls"] != {}]
display(len(sites_dyn))
display(sites_dyn["timeout_dyn"].value_counts())

In [ ]:
# Early abort stats
dyn_conf_run = pd.DataFrame(conn.select("SELECT run, COUNT(id) from dyn_conf GROUP BY run"))
dyn_conf_run.loc["sum"] = dyn_conf_run.sum()
display(dyn_conf_run)

In [ ]:
# Total time taken
sc["insertion_time"].max() - sc["insertion_time"].min() 

## Login detection preparation

In [ ]:
# Limit to first 100 unique sites!
sc = sc.iloc[:100]

In [ ]:
# Convert data (one entry for every confirmed URL)
conf = sc.loc[sc["confirmed_urls"].str.len() != 0]
confs_raw = pd.DataFrame()
for row in conf[["confirmed_urls", "site", "rank"]].iterrows():
    row = row[1]
    site = row["site"]
    rank = row["rank"]
    for state, df in row["confirmed_urls"].items():
        new = pd.DataFrame.from_dict(df)
        new["site"] = site
        new["rank"] = rank
        new["state"] = state
        new = new.rename(columns={"0": "observation_methods"})
        confs_raw = pd.concat([confs_raw, new])
display(confs_raw.head())

confs_raw["observation_methods"] = confs_raw["observation_methods"].apply(sorted)
confs_old = confs_raw.copy()
from publicsuffix2 import get_sld
from urllib.parse import urlparse

confs_raw["real_site"] = confs_raw["url"].apply(lambda x: get_sld(urlparse(json.loads(x)).hostname))
confs_raw["same_site"] = confs_raw["site"] == confs_raw["real_site"]

In [ ]:
all_parties = confs_raw
all_parties = all_parties[all_parties.apply(check_5_vals, axis=1)]
all_parties = all_parties.explode("observation_methods")
all_parties["channel"] = all_parties["inc_method"] + "-" + all_parties["observation_methods"]
# Appy pM Heuristic again
all_parties = all_parties.loc[all_parties.apply(stricter_pm, axis=1)]

In [ ]:
# add lax variant of same-site -> same-party
# first: top-level redirects e.g., blogspot.com == blogger.com
# second: same-party/account used: e.g., google.com == youtube.com

with pd.option_context("display.max_colwidth", None):
    display(all_parties.loc[all_parties["site"] != all_parties["real_site"]].groupby(["rank", "site"])["real_site"].unique().to_frame())

In [ ]:
allow_dict = {
    "bbc.com": ["bbc.co.uk"],
    "blogspot.com": ["blogger.com"],
    "businessinsider.com": ["businessinsider.de"],
    "goo.gl": ["google.com", "youtube.com"],
    "google.com.hk": ["google.com"],
    "huffingtonpost.com": ["huffpost.com"],
    "pinterest.com": ["pinterest.de"],
    "steampowered.com": ["steamcommunity.com"],
    "steamcomunity.com": ["steampowered.com"],
    "techcrunch.com": ["yahoo.com"],
    "yelp.com": ["yelp.de"],
    "youtube.com": ["google.com"],
    "zoho.com": ["zoho.eu"],
    "suara.com": ["zonautara.com"],
    "theverge.com": ["voxmedia.com"]
}
# bbc.com allow bbc.co.uk
# blogspot.com allow blogger.com
# businessinsider.com allow businessinsider.de
# goo.gl allow google.com, youtube.com
# google.com.hk allow google.com
# huffingtonpost.com allow huffpost.com
# pinterest.com allow pinterest.de
# steampowered.com allow steamcommunity.com
# techcrunch.com allow yahoo.com
# yelp.com allow yelp.de
# youtube.com allow google.com

def get_first_party(site):
    sites = allow_dict.get(site, []).copy()
    sites.append(site)
    return sites

def check_party(row):
    if row["real_site"] in row["first_party"]:
        return True
    return False

confs_raw["first_party"] = confs_raw["site"].apply(get_first_party)
display(confs_raw["first_party"].astype(str).value_counts())
confs_raw["same_party"] = confs_raw.apply(check_party, axis=1)

In [ ]:
# Switch to stricter heuristic for analysis!
confs_raw = confs_raw[confs_raw.apply(check_5_vals, axis=1)]
# Use even stricter heuristic for pMs:
# one state is only allowed to have a maximum of one observation, this leads to some FNs, but should remove all FPs
confs_raw = confs_raw.loc[confs_raw.apply(stricter_pm, axis=1)]

In [ ]:
print(f"All considered: {sc['site'].nunique()}")
print(f"All vulnerable: {confs_raw['site'].nunique()}")
print(f"Same-site only: {confs_raw.loc[confs_raw['same_site']]['site'].nunique()}")
print(f"Same-party only: {confs_raw.loc[confs_raw['same_party']]['site'].nunique()}")

In [ ]:
# Sites not making the stricter heuristics
set(confs_old["site"].unique()) ^ set(confs_raw["site"].unique())

In [ ]:
# Sites only vulnerable in third-party context
only_third = set(confs_raw["site"].unique()) ^ set(confs_raw.loc[confs_raw['same_party']]['site'].unique())
print(only_third)
confs_raw.loc[confs_raw["site"].isin(only_third)][["site", "real_site"]].drop_duplicates()

In [ ]:
# Sites not vulnerable at all
sc.loc[sc["confirmed_urls"].str.len() == 0]

## Login detection results

In [ ]:
confs = confs_raw
# Confs same-party only
confs = confs.loc[confs["same_party"] == True]

In [ ]:
print("Unique sites", confs["site"].nunique())
print(confs["observation_methods"].value_counts().to_frame().head(10))

In [ ]:
list(confs["url"].unique())

In [ ]:
confs["channel"] = confs["inc_method"] + "-" +  confs["observation_methods"].apply(str)
display(confs["site"].nunique())
display(confs.groupby(["site"])["browser"].unique().apply(sorted).astype(str).to_frame().value_counts().to_frame())
display(confs.groupby(["browser", "state"])["site"].nunique().to_frame())

In [ ]:
# Explode the observation methods to have one row for every observation method
c_exp = confs.explode("observation_methods")
c_exp["channel"] = c_exp["inc_method"] + "-" + c_exp["observation_methods"]

In [ ]:
# Average URLs/site
display(c_exp.groupby("site")["url"].nunique().to_frame().describe())
# Average inc-url-pairs/site
display(c_exp.groupby("site")["opg_url"].nunique().to_frame().describe())

In [ ]:
# Percentage of vulnerable sites
c_exp["site"].nunique()/len(sc)

In [ ]:
# Fancy tables with Sites both browser, only chromium, only firefox, (sorted by sum)
browser_data = {}

for grouping, name in [(["inc_method"], "incs"), (["observation_methods"], "methods"), (["inc_method", "observation_methods"], "channels")]:
    df = c_exp.loc[c_exp["observation_methods"] != "events-fired-all"].groupby(grouping).apply(get_uniques).apply(pd.Series).sort_values("Sum", ascending=False)
    df = df.reset_index().rename(columns={"inc_method": "Inclusion Method", "observation_methods": "Observation Method"})
    if name == "incs":
        df["Inclusion Method"] = df["Inclusion Method"].apply(json.loads)
        df = df.set_index(["Inclusion Method"])
    elif name == "methods":
        df = df.set_index(["Observation Method"])
    else:
        df["Inclusion Method"] = df["Inclusion Method"].apply(json.loads)
        df = df.set_index(["Inclusion Method", "Observation Method"])
    df = df.rename(index={"fetch_response": "fetch-response"})
    browser_data[name] = df

In [ ]:
#for name in ["incs", "methods", "channels"]:
for name in ["channels"]:
    df = browser_data[name][["Both", "Only C", "Only FF", "Sum"]].head(20)
    df.index = pd.MultiIndex.from_tuples([(x[0], x[1].replace('.smooth', '')) for x in df.index]).set_names(['Inclusion Method', 'Observation Method'])
    
    df.columns = pd.MultiIndex.from_arrays([["Vulnerable sites", "Vulnerable sites", "Vulnerable sites", "Vulnerable sites"], ["Both", "Only Chromium", "Only Firefox", "Sum"]])
    display(df)
    latex_table = df.style.to_latex(hrules=True, multicol_align="c")
    with open(f"res/paper_login_{name}.tex", "w") as f:
        f.write(latex_table)

In [ ]:
#for name in ["incs", "methods", "channels"]:
for name in ["channels"]:
    df = browser_data[name][["Both", "Only C", "Only FF", "Sum"]]
    print(len(df))
    df.index = pd.MultiIndex.from_tuples([(x[0], x[1].replace('.smooth', '')) for x in df.index]).set_names(['Inclusion Method', 'Observation Method'])
    
    df.columns = pd.MultiIndex.from_arrays([["Vulnerable sites", "Vulnerable sites", "Vulnerable sites", "Vulnerable sites"], ["Both", "Only Chromium", "Only Firefox", "Sum"]])
    display(df)
    latex_table = df.style.to_latex(hrules=True, multicol_align="c")
    with open(f"res/paper_login_{name}_full.tex", "w") as f:
        f.write(latex_table)

In [ ]:
df.to_csv("res/paper_login_channels_full.csv")

In [ ]:
print(f"{len(df.loc[df[('Vulnerable sites', 'Only Chromium')] > 0])} working channels in chrome login")
print(f"{len(df.loc[df[('Vulnerable sites', 'Only Firefox')] > 0])} working channels in firefox login")

In [ ]:
with pd.option_context("display.max_rows", 84):
    display(browser_data["channels"][["Both", "Only C", "Only FF", "Sum"]])

### Login vs non-login URLs!

In [ ]:
site_dict = {}
for row in sites.iterrows():
    row = row[1]
    site = row["site"]
    url_dict = {}
    for entry in row["urls"]:
        entry["visited"] = True
        entry["login"] = False
        url_dict[entry["url"]] = entry
        if entry["link"] and entry["request"]:
            print(entry)
    for entry in row["login_urls"]:
        entry["login"] = True
        if url_dict.get(entry["url"], None):
            entry["visited"] = True
        else:
            entry["visited"] = False
        url_dict[entry["url"]] = entry    
    site_dict[site] = url_dict

In [ ]:
def get_source(row):
    site = row["site"]
    url = json.loads(row["url"])
    entry = site_dict[site][url]
    # The later two should be redirects caused by visiting the first one
    if url in [f"https://{site}/", f"http://{site}/", f"https://www.{site}/"]:
        return "hompage"  # "homepage"
    if entry["request"]:
        return "request"
    if entry["link"]:
        return "link"
    else:
        return "invalid"
confs["source"] = confs[["site", "url"]].apply(get_source, axis=1)

In [ ]:
def get_state(row):
    site = row["site"]
    url = json.loads(row["url"])
    entry = site_dict[site][url]
    login = entry["login"] 
    visited = entry["visited"]
    if login and visited:
        return "login and visited"
    elif login:
        return "login"
    elif visited:
        return "visited"
    else:
        raise Exception("Invalid!")
confs["state_source"] = confs[["site", "url"]].apply(get_state, axis=1)

In [ ]:
display(confs["state_source"].value_counts())
display(confs.groupby("state_source")["site"].nunique())
display(confs.groupby("site")["state_source"].unique().reset_index()["state_source"].apply(sorted).astype(str).value_counts())

display(confs[["same_site", "state_source"]].value_counts().to_frame())
confs["state_source_site"] = confs["same_party"].apply(str) + "-" + confs["state_source"]
source_table = confs.groupby("site")["state_source_site"].unique().reset_index()["state_source_site"].apply(sorted).astype(str).value_counts().to_frame()
display(source_table)

In [ ]:
display(confs.groupby("site")["state_source"].unique().reset_index()["state_source"].apply(sorted).astype(str).value_counts())
login_sites = confs.groupby("site")["state_source"].unique().astype(str).to_frame()
display(login_sites.head())
login_sites = login_sites.loc[login_sites["state_source"] == "['login']"].index
login_urls = confs.loc[confs["site"].isin(login_sites)][["site", "url"]].drop_duplicates()

def left_align(df):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )
    return left_aligned_df

# Sites only vulnerable to Login URLs, check that URLs are not session URLs
# No session URLs
with pd.option_context("display.max_colwidth", None):
    display(left_align(login_urls.reset_index()))
    
# All vulnerable URLs found in login mode
# Some might be session URLs, but does not matter as other vulnerable URLs exist on the site
with pd.option_context("display.max_colwidth", None):
    display(left_align(confs.loc[confs["state_source"] == "login"][["site", "url"]].drop_duplicates().reset_index()))

### Link vs requests

In [ ]:
display(confs["source"].value_counts())
display(confs.groupby("source")["site"].nunique())
display(confs.groupby("site")["source"].unique().reset_index()["source"].apply(sorted).astype(str).value_counts())

display(confs[["same_site", "source"]].value_counts().to_frame())
confs["source_site"] = confs["same_site"].apply(str) + "-" + confs["source"]
source_table = confs.groupby("site")["source_site"].unique().reset_index()["source_site"].apply(sorted).astype(str).value_counts().to_frame()
display(source_table)

In [ ]:
def split_data(row):
    data = row["index"][1:-1]
    third_party = []
    first_party = []
    for entry in data.split(", "):
        party, source = entry[1:-1].split("-")
        if party == "False":
            third_party.append(source)
        else:
            first_party.append(source)
    return {"First-Party": sorted(first_party), "Third-Party": sorted(third_party), "Sites": row["source_site"]}
st = source_table.reset_index().apply(split_data, axis=1, result_type="expand").astype(str)
st["First-Party"] = st["First-Party"].apply(lambda x: x.replace("[", "").replace("]", "").replace("'", ""))
st["Third-Party"] = st["Third-Party"].apply(lambda x: x.replace("[", "").replace("]", "").replace("'", ""))
st = st.set_index(["First-Party", "Third-Party"])
display(st)
latex_table = st.head(10).style.to_latex()
with open(f"res/paper_third_login_source.tex", "w") as f:
    f.write(latex_table)

## Third-parties
- third-parties (vs first parties)

In [ ]:
all_parties = confs_raw.explode("observation_methods")
all_parties["channel"] = all_parties["inc_method"] + "-" + all_parties["observation_methods"]

In [ ]:
display(all_parties["site"].nunique())
display(all_parties.groupby("same_site")["site"].nunique())
display(all_parties.groupby("site")["same_site"].unique().reset_index()["same_site"].apply(sorted).astype(str).value_counts())

In [ ]:
# Third-parties that occur often

# A lot of cookie syncing
# Many popular third-parties
third_parties = all_parties.loc[all_parties["same_site"] == False].rename(columns={"real_site": "Third-Party"}).groupby("Third-Party")["site"].nunique().to_frame().sort_values("site", ascending=False)
display(third_parties.head(10))
latex_table = third_parties.head(10).style.to_latex()
with open(f"res/paper_third_login_popular.tex", "w") as f:
     f.write(latex_table)
display(third_parties.describe())